In [1]:
from pyscipopt import Model, Eventhdlr, SCIP_EVENTTYPE,SCIP_HEURTIMING
import pyscipopt
import sys
from tqdm import tqdm
import random
import pandas as pd

In [2]:
class FixedVarsAtNode(Eventhdlr):
    """PySCIPOpt Event handler to write fixed vars of each node to a text file."""

    def __init__(self):
        
        #define the path of file
        # self.output_var_info_file = "var_info/var_info.txt"
        
        #others
        self.orgin_sol_info = pd.DataFrame(columns=['Node', 'ObjVal'])

        self.count=0
        self.count_limit=200000

        self.batch_size = 5000  # 设置每个文件记录的节点数
        self.batch_number = 0  # 当前文件的批次号
        

    def eventexec(self, event):
        if self.count < self.count_limit:
            
            obj_value_no_heuristic = self.model.getObjVal() 
            info_add_df ={'Node': self.count, 'ObjVal': obj_value_no_heuristic}
            self.orgin_sol_info = self.orgin_sol_info.append(info_add_df,ignore_index=True)

            # 判断是否达到记录的节点数
            if self.count % self.batch_size == 0:
                # 写入文件
                self.write_2_file(f'DATA/ip002_output/origin/ip002_output_{self.batch_number}')
                self.batch_number += 1
                # 创建一个新的DataFrame
                self.orgin_sol_info = pd.DataFrame(columns=['Node', 'ObjVal'])

            self.count += 1
        else:
            pass

         
      
    def write_2_file(self , filename):
        #输出为csv文件
        file_name = filename +"_y_info.csv"
        self.orgin_sol_info.to_csv(file_name, index=False)
        

    def eventinit(self):
        self.model.catchEvent(SCIP_EVENTTYPE.NODESOLVED, self)

    def eventexit(self):
        self.model.dropEvent(SCIP_EVENTTYPE.NODESOLVED, self)


In [3]:
if __name__ == "__main__":
    # Specify the output file path
    #output_var_info_file = "var_info.txt"
    
    test = pyscipopt.scip.Model()
    test.setIntParam("heuristics/rins/freq",-1)
    test.readProblem("/Users/oukeikou/Desktop/sunruoyao/DATA/easy-sample/gen-ip002.mps")
    print("read done")


    # Create and add event handler with the specified output file
    eventhdlr = FixedVarsAtNode()
    test.includeEventhdlr(eventhdlr, "FixedVarsAtNode", "Python event handler to write fixed variables after each solved node")

    # Optimize the problem
    test.optimize()
    print("optimized")
    
    # eventhdlr.write_2_file('markshare_4_0')

read doneoriginal problem has 41 variables (0 bin, 41 int, 0 impl, 0 cont) and 24 constraints

feasible solution found by trivial heuristic after 0.0 seconds, objective value 0.000000e+00
presolving:
   (0.0s) running MILP presolver
   (0.0s) MILP presolver found nothing
   (0.0s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.0s) no symmetry present
presolving (1 rounds: 1 fast, 1 medium, 1 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 41 variables (0 bin, 41 int, 0 impl, 0 cont) and 24 constraints
     24 constraints of type <linear>
Presolving Time: 0.00
transformed 1/1 original solutions to the transformed problem space

 time | node  | left  |LP iter|LP it/n|mem/heur|mdpt |vars |cons |rows |cuts |sepa|confs|strbr|  dualbound   | primalbound  |  gap   | compl. 
i 0.0s|     1 |    

FileNotFoundError: [Errno 2] No such file or directory: 'DATA/ip002_output/origin/ip002_output_0_y_info.csv'

Exception ignored in: 'pyscipopt.scip.PyEventExec'
Traceback (most recent call last):
  File "/var/folders/ld/_0mnv9fn0mggqhhlmyx266v80000gn/T/ipykernel_40453/2209914688.py", line 29, in eventexec
  File "/var/folders/ld/_0mnv9fn0mggqhhlmyx266v80000gn/T/ipykernel_40453/2209914688.py", line 43, in write_2_file
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/generic.py", line 3463, in to_csv
    return DataFrameRenderer(formatter).to_csv(
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/io/formats/format.py", line 1105, in to_csv
    csv_formatter.save()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/io/formats/csvs.py", line 237, in save
    with get_handle(
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/io/common.py", line 701, in get_handle
    handle = open(
FileNotFoundError: [Errno 2] No such file o

Exception: SCIP: unspecified error!